In [1]:
import numpy as np
import itertools
import functools

In [2]:
def create_nonlin_dataset(n):
    X = np.random.uniform(0,10,n)
    
    y = 3 - 2*X + X**2
    
    return X,y

In [3]:
X,y = create_nonlin_dataset(100)

In [4]:
X.shape,y.shape

((100,), (100,))

In [7]:
def polynomial_transform(x,degree):
    def get_combinations(x,degree):
        return itertools.combinations_with_replacement(x,degree)
    
    def compute_new_features(items):
        return functools.reduce(lambda x,y:x*y,items)
    
    x = np.array(x)
    x = x.reshape(-1,1)
    
    x_t = np.transpose(x)
    
    features = [np.ones(len(x))]
    
    for degree in range(1,degree+1):
        for items in get_combinations(x_t,degree):
            features.append(compute_new_features(items))
            
    return np.transpose(features)

In [9]:
polynomial_transform([2,3],3)

array([[ 1.,  2.,  4.,  8.],
       [ 1.,  3.,  9., 27.]])

In [23]:
X_poly = polynomial_transform(X,2)

In [80]:
class PolynomialRegression():
    def predict(self,X):
        return X@self.w
    
    def loss(self,X,y,lr=0.05):
        e = self.predict(X) - y
        error = (1/2)*(np.transpose(e)@e)
        ridge_err = (lr/2)*(np.transpose(self.w)@self.w)
        lasso_err = (lr/2)*np.sum(abs(self.w))
        
        if ridge_err>=lasso_err:
            return error + ridge_err
        else:
            return error + lasso_err
        
        
    def calculate_gradient(self,X,y):
        return np.transpose(X)@(self.predict(X) - y)
    
    def update_weights(self,lr,grad):
        return lr*grad
    
    def gradient_descent(self,X,y,lr,epochs=1000):
        self.w = np.zeros(X.shape[-1])
        self.w_all = []
        self.errors_all = []
        
        for _ in range(epochs):
            self.w_all.append(self.w)
            self.errors_all.append(self.loss(X,y))
            
            dJdw = self.calculate_gradient(X,y)
            self.w -= self.update_weights(lr,dJdw)
            
        return self.w_all,self.errors_all
    
    
    def mbgd(self,X,y,lr,batch_size,epochs=100):
        self.w = np.zeros(X.shape[-1])
        self.w_all = []
        self.errors_all = []
        
        for _ in range(epochs):
            shuffled_index = np.random.permutation(X.shape[0])
            X_shuffled = X[shuffled_index]
            y_shuffled = y[shuffled_index]
            
            for i in range(0,X.shape[0],batch_size):
                xi = X_shuffled[i:i+batch_size]
                yi = y_shuffled[i:i+batch_size]
                self.w_all.append(self.w)
                self.errors_all.append(self.loss(xi,yi))
            
                dJdw = (2/batch_size)*self.calculate_gradient(xi,yi)
                
                self.w -= self.update_weights(lr,dJdw)
                
            return self.w_all,self.errors_all
            
        
        

In [81]:
poly_reg = PolynomialRegression()

In [82]:
weights,errors = poly_reg.gradient_descent(X_poly,y,0.000001)

In [83]:
weights[-1]

array([-0.01857503, -0.04973611,  0.80755175])

In [84]:
errors[-1]

93.22056814439819

In [95]:
weights_poly,errors_poly = poly_reg.mbgd(X_poly,y,0.0001,1)

In [96]:
weights_poly[-1]

array([0.00791643, 0.07440758, 0.7815299 ])

In [97]:
errors_poly[-1]

1.2634283726862545

In [98]:
y_pred = poly_reg.predict(X_poly)

In [99]:
y_pred

array([2.56159331e+00, 1.57154047e+01, 5.44200475e+00, 4.78720762e+01,
       3.43770248e+00, 1.11359107e+01, 1.83991536e+01, 7.36161058e+01,
       3.70106029e+01, 1.02550718e+00, 5.96632737e+00, 5.98843940e+00,
       6.64007409e+01, 1.74221039e+01, 5.83734299e-02, 4.82085325e+00,
       2.87445668e+01, 1.73471502e-01, 3.83435830e+01, 5.35073636e+01,
       2.99379662e+00, 4.15997440e+01, 1.60728041e+00, 6.69165615e+00,
       4.56272624e+01, 4.50203051e-01, 4.65617627e+00, 7.50260524e+01,
       1.61121620e+00, 3.65163757e+01, 5.67745927e-01, 1.97525930e+01,
       6.63596093e+01, 4.69700748e+00, 8.06667382e-01, 6.54384345e+01,
       1.19598613e+01, 5.87843328e+01, 4.74056191e+00, 4.62558028e+01,
       6.73663824e+01, 6.81905147e+00, 3.17080133e+01, 2.22389361e+01,
       1.62048243e+01, 1.47325796e+01, 7.45424053e+01, 4.98081602e+01,
       4.74639572e+01, 2.77048674e+01, 2.23083510e+00, 1.52939400e+01,
       2.38897201e+00, 1.71004117e+01, 2.03234867e+01, 3.03149111e+01,
      

ModuleNotFoundError: No module named 'matplotib'